## Importing all the needed Libraries

In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import datetime
import itertools

C:\Users\Dhanesh\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Profile Class with some validations

In [49]:
class Profile:
    # instance attributes
    class_counter= 1
    def __init__(self,email_field, first_name, last_name, date_of_birth = None, class_year= None):
        
        error = ""
        if first_name == '' or last_name== '' or email_field=='':
            error+= "First Name, Last Name and email cannot be empty \n"
        elif(type(first_name)!=str or type(last_name)!=str):
            error += "First Name and Last Name should be string \n"
        
        if len(error):
            print(error)
        else:
            self.id= Profile.class_counter
            Profile.class_counter += 1
            self.first_name = first_name
            self.last_name = last_name
            self.data_of_birth = date_of_birth
            self.class_year = class_year
            self.email_field = email_field
            print('Profile {} Record Entered Successfully!'.format(self.id))
    
    
        

## Method for Duplicate Finder 

In [50]:
def duplicate_finder_helper(profile1, profile2, fields) -> dict:
    mandatory_fields = ['first_name', 'last_name', 'email_field']
    non_mandatory_fields = ['data_of_birth', 'class_year']
    total_score=0
    matching_attributes = [] # -> empty  list where we will collect all the matching fields whose score increase
    nonmatching_attributes = [] # -> empty list where we collect all the fields which are in the fields but does'nt contribute by matching
    ignored_attributes = [f for f in mandatory_fields if f not in fields] # -> Fields which need to be ignored as they are not passed in fields arguments
    name_string_attributes = [f for f in mandatory_fields if f in fields] #-> Mandatory fields which are present in Fields argument
    fuzz_string1 = ''
    fuzz_string2 = ''
    for nms in name_string_attributes:
        fuzz_string1 += getattr(profile1,nms)
        fuzz_string2 += getattr(profile2,nms)
    match_score = fuzz.ratio(fuzz_string1, fuzz_string2) # Calculating fuzz percentage using name_string_attributes
    
    if(match_score > 80):
        matching_attributes = name_string_attributes
        total_score += 1 
    else:
        nonmatching_attributes = name_string_attributes
        
    for field in non_mandatory_fields: # -> Calculating score for non Mandatory Fields.
        if field in fields:
            profile1_field = getattr(profile1, field)
            profile2_field = getattr(profile2, field)
            if(profile1_field!=None and profile2_field!=None):
                if(profile1_field == profile2_field):
                    total_score+=1
                    matching_attributes.append(field)
                else:
                    total_score-=1
                    nonmatching_attributes.append(field)
            else:
                ignored_attributes.append(field)
                
    for f in non_mandatory_fields:
        if f not in fields:
            ignored_attributes.append(f)
                
    ans = dict() #> Contains two KEYs 1. Resultant String as Required in outpu and the score gained
    matching_att_str = 'None' if len(matching_attributes)==0 else ', '.join(str(ma) for ma in matching_attributes)
    nonmatching_att_str = 'None' if len(nonmatching_attributes)==0 else ', '.join(str(nma) for nma in nonmatching_attributes)
    ignored_att_str = 'None' if len(ignored_attributes)==0 else ', '.join(str(ia) for ia in ignored_attributes)
    res = "profile 1, profile 2, total match score : {}, matching_attributes: {}, non_matching_attributes: {}, ignored_attributes: {}".format(total_score, matching_att_str, nonmatching_att_str, ignored_att_str)
    ans['res_string'] = res
    ans['score'] = total_score
    return ans
        

In [51]:
duplicateDF = pd.read_csv('database.csv', index_col=[0]);

In [52]:
def duplicate_finder(profiles, fields, duplicateDF)->None: # -> this function computes score for all possible pairs without repetetion
    n = len(profiles)
    for i in range(n-1):
        for j in range(i+1, n):
            #             print(profiles[i], profiles[j])
            resDict = duplicate_finder_helper(profiles[i]['obj'], profiles[j]['obj'], fields)
            score = resDict['score']
            if score > 1:
                # We are supposed to add the profiles which are having score above 1
                profiles[i]['dict']['id'] = profiles[i]['obj'].id
                profiles[j]['dict']['id'] = profiles[j]['obj'].id
#                 
                dict1 = {'id': profiles[i]['dict']['id'], 'first_name': profiles[i]['dict']['first_name'], 'last_name': profiles[i]['dict']['last_name'], 'email_field':  profiles[i]['dict']['email_field'],'date_of_birth': profiles[i]['dict']['date_of_birth'],'class_year':profiles[i]['dict']['class_year']   }
                dict2 = {'id': profiles[j]['dict']['id'], 'first_name': profiles[j]['dict']['first_name'], 'last_name': profiles[j]['dict']['last_name'], 'email_field':  profiles[j]['dict']['email_field'],'date_of_birth': profiles[j]['dict']['date_of_birth'],'class_year':profiles[j]['dict']['class_year']   }
                df = pd.DataFrame([dict1, dict2], columns=['first_name', 'last_name', 'email_field', 'data_of_birth', 'class_year'])
                duplicateDF = duplicateDF.append(df, ignore_index=True)
#                 print(dict1, dict2)
#                 print(df.head())
                print('adding to DB')
            print(resDict['res_string'])
            

In [53]:

p1 = {'first_name': 'Dhanesh', 'last_name': 'Phulphagar', 'email_field': 'pdhanesh34@gmail.com', 'date_of_birth' :'1999/06/29', 'class_year': None}
p2 = {'first_name': 'Dhanesh', 'last_name': 'Phulpagar', 'email_field': 'pdhanesh343@gmail.com', 'date_of_birth' :'1999/06/29', 'class_year': 2019}
p3 = {'first_name': 'Danesh', 'last_name': 'Fulpagar', 'email_field': 'pdhanesh@gmail.com', 'date_of_birth' :'1999/06/29', 'class_year': 2019}

p1_obj = Profile(**p1)
p2_obj = Profile(**p2)
p3_obj = Profile(**p3)

p1_dict = dict( {'obj': p1_obj, 'dict': p1})
p2_dict = dict( {'obj': p2_obj, 'dict': p2})
p3_dict = dict( {'obj': p3_obj, 'dict': p3})


Profile 1 Record Entered Successfully!
Profile 2 Record Entered Successfully!
Profile 3 Record Entered Successfully!


In [54]:


duplicate_finder([p1_dict, p2_dict], ['first_name', 'last_name', 'email_field', 'data_of_birth', 'class_year'], duplicateDF)

adding to DB
profile 1, profile 2, total match score : 2, matching_attributes: first_name, last_name, email_field, data_of_birth, non_matching_attributes: None, ignored_attributes: class_year


C:\Users\Dhanesh\AppData\Local\Temp\ipykernel_10600\365054494.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  duplicateDF = duplicateDF.append(df, ignore_index=True)


In [55]:
database = pd.read_csv('database.csv', index_col=[0])
database

,first_name,last_name,email_field,data_of_birth,class_year
0,Dhanesh,Phulphagar,pdhanesh34@gmail.com,NaN,NaN
1,Dhanesh,Phulpagar,pdhanesh343@gmail.com,NaN,2019.0
